# YOLO Train

## Read data

In [1]:
import os
import numpy as np
from read_tfrecord import get_dataset
from utils import read_anchors

%load_ext autoreload
%autoreload 2

In [2]:
record_path = './data/train.record'
batch_size = 32
dataset = get_dataset(record_path, [416, 416, 3], 2, 3)
batch = dataset.shuffle(10000).batch(batch_size)

### Read anchors

In [3]:
anchors = read_anchors('model/pet_anchors.txt')

## Build new model

In [4]:
from yolo_model import YoloV3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [5]:
yolov3 = YoloV3(input_shape=(416, 416, 3), 
        num_classes=2,
        anchors=anchors,
        training=True
        )

In [6]:
yolov3.model.load_weights('model/yolo_base.h5', by_name=True)

for layer in yolov3.model.layers[:-3]:
    layer.trainable = False
    
yolov3.model.compile(Adam(1e-2),
        loss=[yolov3.yolo_loss(i) for i in range(len(yolov3.anchor_mask))])

yolov3.model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 416, 416, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 416, 416, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v2 (BatchNo (None, 416, 416, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 416, 416, 32) 0           batch_normalization_v2[0][0]     
______________________________________________________________________________________________

## Train

In [7]:
callbacks = [EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min'),
             ModelCheckpoint(filepath='model/yolo_train.h5', monitor='val_loss', verbose=0, save_best_only=True),
             ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)]

In [ ]:
yolov3.model.fit(batch, epochs=5, callbacks=callbacks)

In [19]:
yolov3.model.save_weights('model/yolo_train.h5')